In [2]:
import duckdb as db
import pandas as pd

In [ ]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pyarrow as pa
import polars as pl


csv_file = "../data/analysis_baseball.csv"
parquet_file = "../data/baseball_analysis.parquet"

BLOCK_SIZE = 120_000_000
read_options = pv.ReadOptions(block_size=BLOCK_SIZE)

# Read CSV with schema enforcement
csv_reader = pv.open_csv(csv_file, read_options=read_options)

parquet_writer = None

try:
    for record_batch in csv_reader:
        print(f"Processing batch with {record_batch.num_rows} rows...")

        # Convert to Polars for further cleaning
        df = pl.from_arrow(record_batch)

        table = df.to_arrow()

        if parquet_writer is None:
            parquet_writer = pq.ParquetWriter(
                parquet_file, 
                schema=table.schema, 
                compression="zstd"
            )
        parquet_writer.write_table(table)

finally:
    if parquet_writer:
        parquet_writer.close()

print(f"Successfully converted {csv_file} to {parquet_file}")


Processing batch with 326281 rows...
Processing batch with 325599 rows...
Processing batch with 327491 rows...
Processing batch with 327239 rows...
Processing batch with 330639 rows...
Processing batch with 331593 rows...
Processing batch with 332094 rows...
Processing batch with 331220 rows...
Processing batch with 330506 rows...
Processing batch with 327535 rows...
Processing batch with 327859 rows...
Processing batch with 325406 rows...
Processing batch with 325846 rows...
Processing batch with 325949 rows...
Processing batch with 325316 rows...
Processing batch with 325436 rows...
Processing batch with 326256 rows...


ArrowInvalid: In CSV column #37: CSV conversion error to null: invalid value '75.38'

In [ ]:
file_path = "../data/concat_baseball.parquet"

result1 = db.query(f"""SELECT pixel_color, COUNT(DISTINCT user_id) AS user_count
                        FROM '{file_path}'
                        WHERE 
                        GROUP BY pixel_color
                        ORDER BY user_count DESC;""")
                      
df1 = result1.to_df()




In [3]:
df = pd.read_csv("../data/concat_baseball.csv")

df.head()

/var/folders/wn/1h4z_5y959g0lxlcphs4q6bc0000gn/T/ipykernel_96576/1907739060.py:1: DtypeWarning: Columns (0,42,76,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/concat_baseball.csv")


: 